# Предобрабока датафрейма

In [ ]:
from pymystem3 import Mystem
import re

m = Mystem()
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

def key_words_in_df_preparation(data=df):
    num_recipes = len(data['list_resipe'])
    list_keywords_dict = []
    list_ingridients_kset = []
    for i in range(0, num_recipes):
        data['list_resipe'][i]=''.join(elem for elem in data['list_resipe'][i])
        res = words_only(data['list_resipe'][i])
        text1 = lemmatize(res)
        text1 = text1.split(' ')
        fdist1 = nltk.FreqDist(text1)
        fdist1 = dict(fdist1)
        list_keywords_dict.append(fdist1)
        
        # ingridient preprocessing
        data['list_ingrid'][i] = ''.join(elem +" " for elem in data['list_ingrid'][i])
        ingridiens = data['list_ingrid'][i]
        res_ingrid = words_only(ingridiens)
        ingrid_lem = lemmatize(res_ingrid)
        ingrid1 = ingrid_lem.split(' ')
        set_ingridients = set(ingrid1)
        list_ingridients_kset.append(set_ingridients)
    data['ingridient_keywords'] = list_ingridients_kset
    data['recipes_keywords'] = list_keywords_dict
    data.to_csv('eda_bulony_new.csv')
    

# Выбираем топ 10 рецептов

In [ ]:
def top10_relevant_recipes(list_ingridients, data):
    
    list_recipes = []
    list_ingridients_common = []
    list_citations_score = []    
    result_list_ingridients = {}
    
    str_ingridients = ''.join(elem + " " for elem in test)
    normailize_ingrid = words_only(str_ingridients)
    normilized_ingrid = lemmatize(normailize_ingrid)
    set_model_result = set(normilized_ingrid.split(' '))
    
    # step by all recipes
    
    for i in range(0, len(data)):
        # number of common ingridients
        ingridients_common = len(set_model_result & data['ingridient_keywords'][i])
        list_ingridients_common.append(ingridients_common)
        # citation score
        citation_score = 0
        for ingrid in list_ingridients:
            try:
                citation_score += data['recipes_keywords'][i][ingrid]
            except KeyError:
                citation_score += 0
        list_citations_score.append(citation_score)
    
    
    top_50_ingrid_common = sorted(range(len(list_ingridients_common)), key=lambda i: list_ingridients_common[i])[-50:]
    top_50_ingrid_common.reverse()
    # Top 50 рецептов с наибольшим совпадением по ингрединтам
    
    dict_top_elements = dict()
    
    for j in range(0, len(top_50_ingrid_common)):
        top_elements_id = top_50_ingrid_common[j]
        dict_top_elements[top_elements_id] = list_ingridients_common[top_elements_id]
    max_value = max(dict_top_elements.values())
    result_list_ingridients = {key:value for key, value in dict_top_elements.items() if value == max_value}
    
    dict_top_10 = dict()
    for j in list(result_list_ingridients.keys()):
        top_10_element_id = list_citations_score[j]
        dict_top_10[top_10_element_id] = list_citations_score[top_10_element_id]
    top_10_citation_common = dict(Counter(dict_top_10).most_common(10))
        
    for item in top_10_citation_common:
        list_recipes.append(data['list_resipe'][item])
    
    return list_recipes